In [5]:
from decouple import config
from datetime import datetime
import requests

In [2]:
assert config('POLYGON_API_KEY', default=None, cast=str) is not None


In [3]:
POLYGON_API_KEY = config('POLYGON_API_KEY', default=None, cast=str)

In [4]:
url2 = "https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/day/2023-01-09/2023-02-10?adjusted=true&sort=asc&apiKey=PtcqMNtGqAQyFqsZyyVhM4ecncCnE8ew"

In [23]:
import requests




response = requests.get(url)

data = response.json()
# print(data['ticker'])


In [9]:
from dataclasses import dataclass
from typing import Literal
from urllib.parse import urlencode

POLYGON_API_KEY = config('POLYGON_API_KEY', default=None, cast=str)

import pytz
from datetime import datetime
def transform_polygon_result(result):
    unix_timestamp = datetime.fromtimestamp(result['t'] / 1000, tz=pytz.timezone('UTC'))
    return {
        'date': result['t'],
        'open_price': result['o'],
        'close_price': result['c'],
        'high_price': result['h'],
        'low_price': result['l'],        
        "number_of_transactions": result['n'],
        'volume': result['v'],
        "volume_weighted_average": result['vw'],
        "time": unix_timestamp
    }


@dataclass
class PolygonAPIClient:
    ticker: str = 'AAPL'
    multiplier: int = 5
    timespan: str = "day"
    from_date: str = "2023-01-09"
    to_date: str = "2023-01-10"
    api_key: str = ""
    adjusted: bool = True
    sort: Literal["asc", "desc"] = "asc"

    def get_api_key(self):
        return self.api_key or POLYGON_API_KEY
        
    def get_headers(self):
        api_key = self.get_api_key()
        return {
            "Authorization": f"Bearer {api_key}"
        }
    
    def get_params(self):
        return {
            "adjusted": self.adjusted,
            "sort": self.sort
        }
    
    def generate_url(self, pass_auth: bool = False):
        path = f'v2/aggs/ticker/{self.ticker}/range/{self.multiplier}/{self.timespan}/{self.from_date}/{self.to_date}?apiKey={POLYGON_API_KEY}'
        url = f"https://api.polygon.io/{path}"
        params = self.get_params()
        encoded_params = urlencode(params)
        url = f"{url}?{encoded_params}"
        if pass_auth:
            url = f"{url}&apiKey={POLYGON_API_KEY}"
        return url

    def fetch_data(self):
        url = self.generate_url()
        headers = self.get_headers()
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()

    def get_stock_data(self):
        data = self.fetch_data()
        return [transform_polygon_result(result) for result in data['results']]


In [11]:
PolygonAPIClient().fetch_data()


{'ticker': 'AAPL',
 'queryCount': 5,
 'resultsCount': 1,
 'adjusted': True,
 'results': [{'v': 333335284.0,
   'vw': 132.1096,
   'o': 130.465,
   'c': 134.76,
   'h': 134.92,
   'l': 128.12,
   't': 1673240400000,
   'n': 2934299}],
 'status': 'OK',
 'request_id': '0234c1865266dd2a8c9f6925053a3a20',
 'count': 1}

In [29]:
result = data['results'][0]


In [36]:
import pytz
from datetime import datetime
def transform_polygon_result(result):
    unix_timestamp = datetime.fromtimestamp(result['t'] / 1000, tz=pytz.timezone('UTC'))
    # print(unix_timestamp)
    return {
        'date': result['t'],
        'open_price': result['o'],
        'close_price': result['c'],
        'high_price': result['h'],
        'low_price': result['l'],        
        "number_of_transactions": result['n'],
        'volume': result['v'],
        "volume_weighted_average": result['vw'],
        "time": unix_timestamp
    }

transform_polygon_result(result)


2023-01-09 05:00:00+00:00


{'date': 1673240400000,
 'open_price': 130.465,
 'close_price': 130.15,
 'high_price': 133.41,
 'low_price': 129.89,
 'number_of_transactions': 645365,
 'volume': 70790813.0,
 'volume_weighted_average': 131.6292,
 'time': datetime.datetime(2023, 1, 9, 5, 0, tzinfo=<UTC>)}